# **Sequence 與 tf.data速度比較**  
實際測試訓練model使用兩種不同的資料載入方式進行訓練，比較差異。  
## 實測環境  
+ CPU: I7-9700K
+ MB: GIGABYTE Z390 GAMING X
+ VGA: ASUS TURBO-RTX2060S-8G-EVO
+ RAM: 32G DDR4 3200
+ SSD: WD SN750 500G

In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print('TensorFlow version:', tf.__version__)
import numpy as np
import imgaug.augmenters as iaa
import os
import pathlib
import time
import math
import cv2

TensorFlow version: 2.2.0


In [2]:
# load dataset
flowers_root = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)
flowers_root = pathlib.Path(flowers_root)

flowers_name = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

images_path = []
for dic in flowers_name:
    for path in os.listdir(flowers_root/dic):
        images_path.append(flowers_root/dic/path)

# data augmentation API
aug = iaa.Sequential(
    [iaa.RandAugment(n=2, m=9),
     iaa.Resize({'longer-side': 224, 'shorter-side': 'keep-aspect-ratio'}),
     iaa.PadToSquare()
    ])

## 使用Sequence訓練

In [3]:
# model
# total data 
total_data = 3670
batch_size = 64

# creat mobilenet model
model = tf.keras.applications.MobileNetV2(weights=None, classes=5)
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=tf.keras.metrics.SparseCategoricalAccuracy())

In [4]:
# Sequence
class flowers(tf.keras.utils.Sequence):
    def __init__(self, flowers_path, batch_size):
        self.path = flowers_path
        self.batch_size = batch_size
    
    def __len__(self):
        return math.ceil(len(self.path) / self.batch_size)
    
    def __getitem__(self, idx):
        batch_path = self.path[idx * self.batch_size: (idx + 1) * self.batch_size]
        
        label = []
        images = []
        
        for i in batch_path:
            label.append(flowers_name.index(i.parent.name))
            img = cv2.imread(str(i))
            img = aug(image=img)
            images.append(img)
        
        label = np.array(label)
        images = np.array(images) / 255
        return images, label
    
    def on_epoch_end(self):
        np.random.shuffle(self.path)

In [5]:
start_time = time.time()
model.fit(flowers(images_path, batch_size), epochs=5)
print('Use times:{:4.4f}'.format(time.time() - start_time))

Epoch 1/5
58/58 [==============================] - 21s 355ms/step - loss: 1.9769 - sparse_categorical_accuracy: 0.1717
Epoch 2/5
58/58 [==============================] - 20s 348ms/step - loss: 1.5124 - sparse_categorical_accuracy: 0.3245
Epoch 3/5
58/58 [==============================] - 20s 345ms/step - loss: 1.2897 - sparse_categorical_accuracy: 0.4490
Epoch 4/5
58/58 [==============================] - 20s 344ms/step - loss: 1.1799 - sparse_categorical_accuracy: 0.5131
Epoch 5/5
58/58 [==============================] - 20s 352ms/step - loss: 1.1075 - sparse_categorical_accuracy: 0.5490
Use times:110.0877


## 使用tf.data訓練

In [6]:
list_ds = tf.data.Dataset.list_files(str(flowers_root/'*/*.jpg'))

def get_label(name):
    label = flowers_name.index(name.numpy().decode('UTF-8'))
    return label

def aug_img(image):
    image = aug(image=image.numpy())
    return image

# 依照路徑資料進行處理
def process_path(path):
    # 取出檔名
    name = tf.strings.split(path, os.sep)[-2]
    # 找出對應的index
    label = tf.py_function(get_label, [name], tf.float32)
    # 載入圖片
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image)
    image = tf.py_function(aug_img, [image], tf.uint8)
    # 轉換成float32以及normalize
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    return image, label

In [7]:
train_data = list_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [8]:
# model
# total data 
total_data = 3670
batch_size = 64

# creat mobilenet model
model = tf.keras.applications.MobileNetV2(weights=None, classes=5)
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=tf.keras.metrics.SparseCategoricalAccuracy())

In [9]:
train_data = train_data.batch(batch_size).repeat().prefetch(tf.data.experimental.AUTOTUNE)

In [10]:
start_time = time.time()
model.fit(train_data, epochs=5, steps_per_epoch=math.ceil(total_data/batch_size))
print('Use times:{:4.4f}'.format(time.time() - start_time))

Epoch 1/5
58/58 [==============================] - 18s 304ms/step - loss: 1.4564 - sparse_categorical_accuracy: 0.3651
Epoch 2/5
58/58 [==============================] - 17s 299ms/step - loss: 1.2566 - sparse_categorical_accuracy: 0.4586
Epoch 3/5
58/58 [==============================] - 18s 304ms/step - loss: 1.2026 - sparse_categorical_accuracy: 0.5038
Epoch 4/5
58/58 [==============================] - 19s 321ms/step - loss: 1.1179 - sparse_categorical_accuracy: 0.5542
Epoch 5/5
58/58 [==============================] - 18s 303ms/step - loss: 1.0401 - sparse_categorical_accuracy: 0.5886
Use times:94.4882


## **總結**  
雖然`Sequence`與`tf.data`都是做相同的事情，但是`tf.data`在這個實驗當中快了一點。  
但是在另一個較舊款的電腦上，實行時間`tf.data`幾乎快了一倍，看來之後必須改用`tf.data`來提升效率。